In [73]:
import numpy as np
import pandas as pd
import sklearn.neighbors
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split,KFold,GridSearchCV, cross_val_score

In [2]:
data = pd.read_csv("C:/Users/hp/Downloads/uber_rides_data.xlsx - sample_train.csv")

In [3]:
data.head()

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [4]:
data.shape

(200000, 8)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ride_id            200000 non-null  int64  
 1   fare_amount        200000 non-null  float64
 2   pickup_datetime    200000 non-null  object 
 3   pickup_longitude   200000 non-null  float64
 4   pickup_latitude    200000 non-null  float64
 5   dropoff_longitude  199999 non-null  float64
 6   dropoff_latitude   199999 non-null  float64
 7   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 12.2+ MB


In [6]:
data['dropoff_longitude'].isnull().sum()

1

In [7]:
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   ride_id            200000 non-null  int64              
 1   fare_amount        200000 non-null  float64            
 2   pickup_datetime    200000 non-null  datetime64[ns, UTC]
 3   pickup_longitude   200000 non-null  float64            
 4   pickup_latitude    200000 non-null  float64            
 5   dropoff_longitude  199999 non-null  float64            
 6   dropoff_latitude   199999 non-null  float64            
 7   passenger_count    200000 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(5), int64(2)
memory usage: 12.2 MB


In [9]:
data.dropna()

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,7.5,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,7.7,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,12.9,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,5.3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,16.0,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,5
...,...,...,...,...,...,...,...,...
199995,42598914,3.0,2012-10-28 10:49:00+00:00,-73.987042,40.739367,-73.986525,40.740297,1
199996,16382965,7.5,2014-03-14 01:09:00+00:00,-73.984722,40.736837,-74.006672,40.739620,1
199997,27804658,30.9,2009-06-29 00:42:00+00:00,-73.986017,40.756487,-73.858957,40.692588,2
199998,20259894,14.5,2015-05-20 14:56:25+00:00,-73.997124,40.725452,-73.983215,40.695416,1


In [10]:
data.describe()

,ride_id,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,2.000000e+05,200000.000000,200000.000000,200000.000000,199999.000000,199999.000000,200000.000000
mean,2.771250e+07,11.359955,-72.527638,39.935885,-72.525292,39.923890,1.684535
std,1.601382e+07,9.901776,11.437787,7.720539,13.117408,6.794829,1.385997
min,1.000000e+00,-52.000000,-1340.648410,-74.015515,-3356.666300,-881.985513,0.000000
25%,1.382535e+07,6.000000,-73.992065,40.734796,-73.991407,40.733823,1.000000
50%,2.774550e+07,8.500000,-73.981823,40.752592,-73.980093,40.753042,1.000000
75%,4.155530e+07,12.500000,-73.967153,40.767158,-73.963659,40.768001,2.000000
max,5.542357e+07,499.000000,57.418457,1644.421482,1153.572603,872.697628,208.000000


In [11]:
data.head()

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,7.5,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,7.7,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,12.9,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,5.3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,16.0,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,5


In [12]:
import pandas as pd
import numpy as np

# Sample dataset (replace this with your actual dataset)
dataset = {
    'pickup_latitude': data['pickup_latitude'],
    'pickup_longitude': data['pickup_longitude'],
    'dropoff_latitude': data['dropoff_latitude'],
    'dropoff_longitude': data['dropoff_longitude']
}

df = pd.DataFrame(data)

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))

    distance = R * c
    return distance

# Calculate distances for all pairs of pickup and dropoff using vectorized operations
df['distance'] = haversine(df['pickup_latitude'], df['pickup_longitude'], df['dropoff_latitude'], df['dropoff_longitude'])

# Calculate the median distance
median_distance = df['distance'].median()
print("Median Haversine distance:", median_distance, "kilometers")
max_distance = df['distance'].max()
print("Maximum Haversine distance:", max_distance, "kilometers")
rides_with_zero_distance = df[df['distance'] == 0.0]

count_rides_with_zero_distance = rides_with_zero_distance.shape[0]

print("Number of rides with 0.0 Haversine distance:", count_rides_with_zero_distance)

# Calculate the mean 'fare_amount' for rides with 0.0 Haversine distance
mean_fare_amount_for_zero_distance_rides = rides_with_zero_distance['fare_amount'].mean()

print("Mean 'fare_amount' for rides with 0.0 Haversine distance:", mean_fare_amount_for_zero_distance_rides)

Median Haversine distance: 2.1209923961833708 kilometers
Maximum Haversine distance: 16409.239135313168 kilometers
Number of rides with 0.0 Haversine distance: 5632
Mean 'fare_amount' for rides with 0.0 Haversine distance: 11.585317826704546


In [13]:
data['fare_amount'].describe()

count    200000.000000
mean         11.359955
std           9.901776
min         -52.000000
25%           6.000000
50%           8.500000
75%          12.500000
max         499.000000
Name: fare_amount, dtype: float64

In [14]:
data[['fare_amount']].idxmax()

fare_amount    170081
dtype: int64

In [15]:
data.iloc[170081]

ride_id                               51151143
fare_amount                              499.0
pickup_datetime      2011-04-10 04:10:00+00:00
pickup_longitude                    -73.968377
pickup_latitude                      40.764602
dropoff_longitude                   -73.968368
dropoff_latitude                       40.7646
passenger_count                              1
distance                               0.00079
Name: 170081, dtype: object

In [16]:
df = pd.DataFrame(data['ride_id'], data['pickup_datetime'])

In [17]:
import pandas as pd

# Sample dataset (replace this with your actual dataset)
data_dict = {
    'ride_id': data['ride_id'],
    'pickup_datetime': data['pickup_datetime']
}

df = pd.DataFrame(data_dict)

# Convert the 'pickup_datetime' column to datetime format
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

# Extract the year from the 'pickup_datetime' column and create a new 'year' column
df['year'] = df['pickup_datetime'].dt.year

# Count the number of rides recorded in the year 2014
rides_2014 = df[df['year'] == 2014]

# Get the count of rides in 2014
count_rides_2014 = rides_2014.shape[0]

print("Number of rides recorded in the year 2014:", count_rides_2014)


Number of rides recorded in the year 2014: 29968


In [18]:
df['quarter'] = df['pickup_datetime'].dt.quarter

# Count the number of rides recorded in the first quarter of 2014
rides_first_quarter_2014 = df[(df['year'] == 2014) & (df['quarter'] == 1)]

# Get the count of rides in the first quarter of 2014
count_rides_first_quarter_2014 = rides_first_quarter_2014.shape[0]

print("Number of rides recorded in the first quarter of 2014:", count_rides_first_quarter_2014)

Number of rides recorded in the first quarter of 2014: 7687


In [19]:
df['day_of_week'] = df['pickup_datetime'].dt.day_name()

# Filter for rides that occurred in September 2010
rides_september_2010 = df[(df['pickup_datetime'].dt.year == 2010) & (df['pickup_datetime'].dt.month == 9)]

# Find the day of the week with the maximum rides
max_day_of_week = rides_september_2010['day_of_week'].mode()[0]

print("Day of the week in September 2010 with the maximum rides:", max_day_of_week)

Day of the week in September 2010 with the maximum rides: Thursday


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [21]:
data['weekday'] = df.pickup_datetime.dt.weekday

In [22]:
data.dropna()

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance,weekday
0,24238194,7.5,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1,1.683323,3
1,27835199,7.7,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,1,2.457590,4
2,44984355,12.9,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,1,5.036377,0
3,25894730,5.3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,3,1.661683,4
4,17610152,16.0,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,5,4.475450,3
...,...,...,...,...,...,...,...,...,...,...
199995,42598914,3.0,2012-10-28 10:49:00+00:00,-73.987042,40.739367,-73.986525,40.740297,1,0.112210,6
199996,16382965,7.5,2014-03-14 01:09:00+00:00,-73.984722,40.736837,-74.006672,40.739620,1,1.875050,4
199997,27804658,30.9,2009-06-29 00:42:00+00:00,-73.986017,40.756487,-73.858957,40.692588,2,12.850319,0
199998,20259894,14.5,2015-05-20 14:56:25+00:00,-73.997124,40.725452,-73.983215,40.695416,1,3.539715,2


## Feature Selection

In [23]:
features =  ['passenger_count',
             'weekday','distance']

target = 'fare_amount'

In [24]:
X, y = data[features], data[target]

In [25]:
spearman_matrix = df.corr(method='spearman')
spearman_matrix

C:\Users\hp\AppData\Local\Temp\ipykernel_1120\3785498557.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  spearman_matrix = df.corr(method='spearman')


,ride_id,year,quarter
ride_id,1.000000,-0.001343,0.001506
year,-0.001343,1.000000,-0.111505
quarter,0.001506,-0.111505,1.000000


In [26]:
abs_corr_values = spearman_matrix.abs()
top_features = abs_corr_values.mean().sort_values().head(6)
top_features

ride_id    0.334283
year       0.370949
quarter    0.371003
dtype: float64

In [27]:
data = data.dropna()

In [47]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.3, random_state = 355)

In [49]:
X_train = X_train.fillna(method = 'ffill')

In [50]:
scalar = StandardScaler()

X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)
X_train.shape

(140000, 3)

In [51]:
split_models = {
    'Random_Forest_1': RandomForestRegressor(),
    'Decision_tree_1': DecisionTreeRegressor(),
    'Linear_Regression_1': LinearRegression(),
    'KNN': KNeighborsRegressor()
}

In [59]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def eval_model(y_true, y_pred, X):
    # Calculate metrics
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    # Calculate the number of observations (n) and predictors (k)
    n = len(y_true)
    k = X.shape[1]
    
    # Calculate the adjusted R-squared (adjusted R2)
    adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - k - 1))
    
    return mae, mse, r2, adjusted_r2

In [60]:
train_test_results = {}

for model_name, model in tqdm(split_models.items(), desc='Training Models'):
    model.fit(X_train, y_train.ravel())

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    trained_data_mae, trained_data_mse, trained_data_r2, trained_data_adjusted_r2 = eval_model(y_train, y_train_pred, X_train)
    test_data_mae, test_data_mse, test_data_r2, test_data_adjusted_r2 = eval_model(y_test, y_test_pred, X_test)

    train_test_results[model_name] = {
        'Train': {
            'Train_MAE': trained_data_mae,
            'Train_R2': trained_data_r2,
            'Train_MSE': trained_data_mse,
            'Train_Adjusted_R2': trained_data_adjusted_r2
        },
        'Test': {
            'Test_MAE': test_data_mae,
            'Test_R2': test_data_r2,
            'Test_MSE': test_data_mse,
            'Test_Adjusted_R2': test_data_adjusted_r2
        }
    }


Training Models: 100%|███████████████████████████████████████████████████████████████████| 4/4 [01:40<00:00, 25.23s/it]


In [62]:
print('='*100)
print('{:<25s} {:<8s} {:<8s} {:<8s} {:<8s} {:<12s}'.format('Model', 'Train_MSE', 'Test_MSE', 'Train_R2', 'Test_R2', 'Train_Adjusted_R2'))
print('='*100)

for model_name, model_results in train_test_results.items():
    print('{:<25s} {:<9.3f} {:<9.3f} {:<9.3f} {:<9.3f} {:<13.3f}'.format(
        model_name,
        model_results['Train']['Train_MSE'],
        model_results['Test']['Test_MSE'],
        model_results['Train']['Train_R2'],
        model_results['Test']['Test_R2'],
        model_results['Train']['Train_Adjusted_R2']
    ))

print('='*100)


Model                     Train_MSE Test_MSE Train_R2 Test_R2  Train_Adjusted_R2
Random_Forest_1           9.286     36.977    0.903     0.639     0.903        
Decision_tree_1           6.106     52.213    0.936     0.491     0.936        
Linear_Regression_1       96.042    102.469   0.001     0.001     0.001        
KNN                       22.460    36.087    0.766     0.648     0.766        


## Cross Validation

In [68]:
scalar_1 = StandardScaler()

X_fold = scalar_1.fit_transform(data[features])
y_fold = data.fare_amount

In [69]:
Kfold_models = {
    'Rnadom_Forest_2': RandomForestRegressor(),
    'Decision_tree_2': DecisionTreeRegressor(),
    'Linear_Regression_2': LinearRegression(),
    'KNN': KNeighborsRegressor()
}

In [74]:
kfold_results = {}

folds = 5
k_folds = KFold(n_splits=folds)

for model_name, model in tqdm(Kfold_models.items(), desc='Training Models'):
    scores = cross_val_score(model, X_fold, y_fold.ravel(), scoring='r2')
    
    kfold_results[model_name] = {
        'Fold_Scores': scores,
        'Average_Score': np.mean(scores)
}

Training Models: 100%|██████████████████████████████████████████████████████████████████| 4/4 [08:19<00:00, 124.99s/it]


In [75]:
print('='*80)
print('{:<25s} {:<35s} {:<12s}'.format('Model', 'R2 Scores', 'Average R2'))
print('='*80)
for model_name, model_results in kfold_results.items():
    scores_str = ', '.join([f'{score:.3f}' for score in model_results['Fold_Scores']])
    print('{:<25s} {:<35s} {:<12.3f}'.format(model_name, scores_str, model_results['Average_Score']))
print('='*80)

Model                     R2 Scores                           Average R2  
Rnadom_Forest_2           0.640, 0.686, 0.705, 0.655, 0.636   0.664       
Decision_tree_2           0.491, 0.533, 0.553, 0.489, 0.486   0.510       
Linear_Regression_2       0.001, 0.001, 0.001, 0.000, 0.001   0.001       
KNN                       0.643, 0.675, 0.686, 0.653, 0.631   0.658       
